<a href="https://www.kaggle.com/code/diamelamine/seneweb?scriptVersionId=113604369" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# SeneWeb par Lamine DIAME & Elhadji LEYE

### par Lamine DIAME & Elhadji LAYE

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('../input/mesarticles/articles_projet.csv')
data

,identifiantArticle,contenuArticle,thematiqueArticle
0,https___actu24.net_13-personnes-convoquees-a-k...,Après l’accalmie de ces derniers jours avec la...,Politique
1,http___sentv.info_actualite-nationale_politiqu...,SENTV.info : L’attribution des marchés des den...,Politique
2,http___www.seneweb.com_news_justice_saccage-de...,Saccage de ses magasins : Auchan traîne en jus...,Justice
3,http___www.seneweb.com_news_politique_issa-sal...,Issa Sall nommé (depuis des mois) ministre con...,Politique
4,http___www.seneweb.com_news_sante_les-vaccins-...,Les Vaccins chinois surs et efficaces Les vacc...,Sante
...,...,...,...
7833,https___www.seneweb.com_news_politique_saccage...,"Bachirou Ba, collectif Aar sunu momel Le coord...",Politique
7834,https___www.seneweb.com_news_politique_report-...,Report des Elections locales Le report des éle...,Politique
7835,https___www.seneweb.com_news_justice_plainte-c...,Plainte contre Prési Cissé et Cie : Dakaractu ...,Justice
7836,https___www.seneweb.com_news_sante_covid-19-5-...,"Covid-19 : 5 décès, 52 nouveaux tests positifs...",Sante


#  Analyse exploratoire des données 

In [3]:
# Veillez ouvrir le fichier exploratoire_des_donnees.html 
# pour voir le résultat  
from pandas_profiling import ProfileReport
prof = ProfileReport(data)
prof.to_file(output_file='exploratoire_des_donnees.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Pré-traitement des données

In [4]:
#Pour chaque feature on affiche son nombre de valeurs manquantes
data.isna().sum()

identifiantArticle     0
contenuArticle        16
thematiqueArticle      0
dtype: int64

In [5]:
# On supprime les articles qui n'ont pas de contenu article
data.dropna(subset=["contenuArticle"], inplace = True)
data

,identifiantArticle,contenuArticle,thematiqueArticle
0,https___actu24.net_13-personnes-convoquees-a-k...,Après l’accalmie de ces derniers jours avec la...,Politique
1,http___sentv.info_actualite-nationale_politiqu...,SENTV.info : L’attribution des marchés des den...,Politique
2,http___www.seneweb.com_news_justice_saccage-de...,Saccage de ses magasins : Auchan traîne en jus...,Justice
3,http___www.seneweb.com_news_politique_issa-sal...,Issa Sall nommé (depuis des mois) ministre con...,Politique
4,http___www.seneweb.com_news_sante_les-vaccins-...,Les Vaccins chinois surs et efficaces Les vacc...,Sante
...,...,...,...
7833,https___www.seneweb.com_news_politique_saccage...,"Bachirou Ba, collectif Aar sunu momel Le coord...",Politique
7834,https___www.seneweb.com_news_politique_report-...,Report des Elections locales Le report des éle...,Politique
7835,https___www.seneweb.com_news_justice_plainte-c...,Plainte contre Prési Cissé et Cie : Dakaractu ...,Justice
7836,https___www.seneweb.com_news_sante_covid-19-5-...,"Covid-19 : 5 décès, 52 nouveaux tests positifs...",Sante


In [6]:
import spacy
from spacy.lang.fr import French
nlp = French()
# Les mots vides de notre jeu de données
french_stopwords = spacy.lang.fr.stop_words.STOP_WORDS

In [7]:
# Définition de Fonction doc_without_stop_word pour le pre-traitement des textes
def doc_without_stop_word(text):
    doc = nlp(text)
    filtered = [str(token.text).lower() for token in doc if token.is_stop==False and token.text.isalpha()==True]
    filtered_text = ' '.join(filtered)
    return filtered_text

In [8]:
# Le contenu sans mots vides
data['text_filtre'] = data['contenuArticle'].apply(lambda x: doc_without_stop_word(x))
data['text_filtre']

0       accalmie derniers jours libération personnes a...
1       attribution marchés denrées aide alimentaire g...
2       saccage magasins auchan traîne justice employé...
3       issa sall nommé mois ministre conseiller tribu...
4       vaccins chinois surs efficaces vaccins laborat...
                              ...                        
7833    bachirou ba collectif aar sunu momel coordonna...
7834    report elections locales report élections loca...
7835    plainte contre prési cissé cie dakaractu final...
7836    décès nouveaux tests positifs patients réa com...
7837    étude américaine qualité sperme dégringole mon...
Name: text_filtre, Length: 7822, dtype: object

In [9]:
# Affichage des thématiques
data.thematiqueArticle.unique()

array(['Politique', 'Justice', 'Sante', 'Sport', 'Economie', 'Education'],
      dtype=object)

In [10]:
# Mappage des thématiques 
data['thematiqueArticle'] = data.thematiqueArticle.map({ 'Politique': 1, 'Justice': 2, 'Sante': 3, 'Sport': 4, 'Economie': 5, 'Education': 6 })
data.head()

,identifiantArticle,contenuArticle,thematiqueArticle,text_filtre
0,https___actu24.net_13-personnes-convoquees-a-k...,Après l’accalmie de ces derniers jours avec la...,1,accalmie derniers jours libération personnes a...
1,http___sentv.info_actualite-nationale_politiqu...,SENTV.info : L’attribution des marchés des den...,1,attribution marchés denrées aide alimentaire g...
2,http___www.seneweb.com_news_justice_saccage-de...,Saccage de ses magasins : Auchan traîne en jus...,2,saccage magasins auchan traîne justice employé...
3,http___www.seneweb.com_news_politique_issa-sal...,Issa Sall nommé (depuis des mois) ministre con...,1,issa sall nommé mois ministre conseiller tribu...
4,http___www.seneweb.com_news_sante_les-vaccins-...,Les Vaccins chinois surs et efficaces Les vacc...,3,vaccins chinois surs efficaces vaccins laborat...


## Division des données en training set et test set

In [11]:
# Division des données en training et test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data['text_filtre'], 
    data['thematiqueArticle'], 
    random_state = 1
)

print("Training dataset: ", X_train.shape[0])
print("Test dataset: ", X_test.shape[0])

Training dataset:  5866
Test dataset:  1956


 # Représentation des textes

 ## Word Count Vectors

#### La bibliothèque de sklearn contient une fonction **CountVectorizer()** qui permet de "vectoriser" un ensemble de textes bruts en prenant en compte un certain nombre de prétraitements (preprocessing) couramment employés. Le CountVectorizer permet de compter le nombre de mots (fréquence des termes).

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
cv = CountVectorizer()

In [14]:
cv.fit(data['text_filtre'])

CountVectorizer()

In [15]:
##Si tu veux observer le vocabulaire decumente la ligne ci-dessous
##cv.vocabulary_

#### On peut maintenant construire la matrice documents * termes (dtm) à l'aide de la fonction **transform()**.

In [16]:
dtm = cv.transform(data['text_filtre'])
dtm.shape

(7822, 56415)

#### Nous pouvons créer un DataFrame pour représenter les textes et les attributs sous forme de tableau à 2 dimensions

In [17]:
terms = cv.get_feature_names()
df_cv = pd.DataFrame(dtm.toarray(), columns=terms)
df_cv

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,aaas,aamadu,aan,aanholt,aar,aaron,aarons,aastou,aautour,...,ɗe,ɗee,ɗemngal,ɗen,ɗereeji,ɗi,ɗiɗi,ɗowgol,ɗum,ʺla
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7817,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7818,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7819,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7820,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## TF–IDF Vectors

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### la classe **TfidfTransformer** permet de transformer une matrice de comptage en une représentation tf ou tf-idf normalisée. Ici, nous allons l'appliquer sur le comptage de mots calculés précédemment.

In [19]:
vectorizer = TfidfVectorizer(sublinear_tf=True,min_df=5,max_df=0.95)

In [20]:
tfidf_dtm=vectorizer.fit_transform(data['text_filtre'])
tfidf_dtm.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
termsVect = vectorizer.get_feature_names()
tdidf_df = pd.DataFrame(tfidf_dtm.toarray(), columns=termsVect)
tdidf_df

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aar,ab,aba,ababacar,abaisser,abandon,abandonnant,abandonne,abandonner,abandonné,...,ôter,œil,œuf,œuvrant,œuvre,œuvrent,œuvrer,œuvres,œuvrons,œuvré
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7817,0.234808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.052305,0.0,0.0,0.0,0.0,0.0
7818,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
7819,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
7820,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


# Modèles

In [22]:
x_train = vectorizer.fit_transform(X_train)

In [23]:
terms = vectorizer.get_feature_names()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [24]:
df_train = pd.DataFrame(x_train.toarray(), columns=terms)
df_train

,aar,ab,ababacar,abaisser,abandon,abandonnant,abandonne,abandonner,abandonné,abandonnée,...,ñi,ñoo,ñu,ôter,œil,œuvrant,œuvre,œuvrent,œuvrer,œuvres
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.068306,0.0,0.0,0.0
5862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [25]:
x_test = vectorizer.transform(X_test)

In [26]:
terms = vectorizer.get_feature_names()
df_test = pd.DataFrame(x_test.toarray(), columns=terms)
df_test

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aar,ab,ababacar,abaisser,abandon,abandonnant,abandonne,abandonner,abandonné,abandonnée,...,ñi,ñoo,ñu,ôter,œil,œuvrant,œuvre,œuvrent,œuvrer,œuvres
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier # Import Decision Tree Classifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

#### Nous créons un modèle d'arbre de décision et estimons les paramètres du modèle.

In [28]:

dt_clf = RandomForestClassifier(max_depth=20, random_state=1)

dt_clf = dt_clf.fit(x_train,y_train)

#### Après l'apprentissage, nous pouvons prédire le test maintenant.

In [29]:
y_pred = dt_clf.predict(x_test)

#### Nous comparons les classes prédites par le système aux classes réelles et affichons d'abord l'exactitude.

In [30]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8016359918200409


#### On pouvons afficher également un rapport d'evaluation plus détaillé avec les valeurs de précision, rappel et f-score.

In [31]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.70      0.97      0.81       852
           2       1.00      0.36      0.53       167
           3       0.92      0.84      0.88       394
           4       0.99      0.92      0.95       319
           5       1.00      0.10      0.17       126
           6       0.94      0.46      0.62        98

    accuracy                           0.80      1956
   macro avg       0.93      0.61      0.66      1956
weighted avg       0.85      0.80      0.77      1956



##  Support Vector Machine

In [32]:
from sklearn import svm #  Support Vector Machine

#### Nous créons un modèle de svm et estimons les paramètres du modèle.

In [33]:
svm_clf = svm.SVC()
dsvm_clf = svm_clf.fit(x_train,y_train)

#### Après l'apprentissage, nous pouvons prédire le test maintenant.

In [34]:
y_pred = svm_clf.predict(x_test)

#### Nous comparons les classes prédites par le système aux classes réelles et affichons d'abord l'exactitude.

In [35]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9079754601226994


#### On pouvons afficher également un rapport d'evaluation plus détaillé avec les valeurs de précision, rappel et f-score

In [36]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.86      0.96      0.91       852
           2       0.94      0.86      0.90       167
           3       0.95      0.91      0.93       394
           4       0.98      0.97      0.98       319
           5       0.91      0.57      0.70       126
           6       0.90      0.82      0.86        98

    accuracy                           0.91      1956
   macro avg       0.92      0.85      0.88      1956
weighted avg       0.91      0.91      0.91      1956



## K Nearest Neighbors

In [37]:
from sklearn.neighbors import KNeighborsClassifier #Import NearestNeighbors

#### Nous créons un modèle de NearestNeighbors et estimons les paramètres du modèle.

In [38]:
knn_clf = KNeighborsClassifier(n_neighbors=7)
dtknn_clf = knn_clf.fit(x_train,y_train)

#### Après l'apprentissage, nous pouvons prédire le test maintenant.

In [39]:
y_pred = knn_clf.predict(x_test)

#### Nous comparons les classes prédites par le système aux classes réelles et affichons d'abord l'exactitude.

In [40]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8757668711656442


#### On pouvons afficher également un rapport d'evaluation plus détaillé avec les valeurs de précision, rappel et f-score

In [41]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.86      0.89      0.88       852
           2       0.84      0.83      0.83       167
           3       0.88      0.87      0.88       394
           4       0.98      0.96      0.97       319
           5       0.76      0.63      0.69       126
           6       0.82      0.90      0.86        98

    accuracy                           0.88      1956
   macro avg       0.86      0.85      0.85      1956
weighted avg       0.88      0.88      0.87      1956



## Multinomial Naïve Bayes

In [42]:
from sklearn.naive_bayes import MultinomialNB

#### Nous créons un modèle de Multinomial Naïve Bayes et estimons les paramètres du modèle.

In [43]:
mnn_clf = MultinomialNB()
dtmnn_clf = mnn_clf.fit(x_train,y_train)

#### Après l'apprentissage, nous pouvons prédire le test maintenant.

In [44]:
y_pred = mnn_clf.predict(x_test)

#### Nous comparons les classes prédites par le système aux classes réelles et affichons d'abord l'exactitude.

In [45]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7571574642126789


#### On pouvons afficher également un rapport d'evaluation plus détaillé avec les valeurs de précision, rappel et f-score

In [46]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.66      0.94      0.78       852
           2       1.00      0.40      0.57       167
           3       0.85      0.78      0.81       394
           4       0.99      0.92      0.95       319
           5       1.00      0.04      0.08       126
           6       1.00      0.04      0.08        98

    accuracy                           0.76      1956
   macro avg       0.92      0.52      0.55      1956
weighted avg       0.82      0.76      0.71      1956



## Multinomial Logistic Regression

In [47]:
from sklearn.linear_model import LogisticRegression

#### Nous créons un modèle de Multinomial Logistic Regression et estimons les paramètres du modèle.

In [48]:
log_reg_clf = LogisticRegression(solver='newton-cg', multi_class='multinomial')
dtlog_clf=log_reg_clf.fit(x_train,y_train)

#### Après l'apprentissage, nous pouvons prédire le test maintenant.

In [49]:
y_pred = log_reg_clf.predict(x_test)

#### Nous comparons les classes prédites par le système aux classes réelles et affichons d'abord l'exactitude.

In [50]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8870143149284253


#### On pouvons afficher également un rapport d'evaluation plus détaillé avec les valeurs de précision, rappel et f-score

In [51]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.84      0.94      0.89       852
           2       0.92      0.81      0.86       167
           3       0.92      0.87      0.89       394
           4       0.98      0.96      0.97       319
           5       0.84      0.56      0.67       126
           6       0.91      0.81      0.85        98

    accuracy                           0.89      1956
   macro avg       0.90      0.82      0.86      1956
weighted avg       0.89      0.89      0.88      1956

